# Hourly Trend Analysis

This notebook analyzes weather data trends by comparing:
- **Last completed hour** vs **Previous hour**
- Shows signed deltas for temperature, humidity, and pressure
- Formatted as: `+1.2°C`, `-3.0%`, `+0.8 hPa`

In [1]:
import asyncio
import pandas as pd
from prisma import Prisma
from datetime import datetime, timedelta
import pytz

In [ ]:
import os
from pathlib import Path

# Load environment variables from .env file
from dotenv import load_dotenv

# Get the Data directory path
data_dir = Path('/home/aneesh/Desktop/Code/SkyDelta/Data')
env_path = data_dir / '.env'

# Load the .env file
load_dotenv(env_path)

# Verify DATABASE_URL is loaded
if not os.getenv('DATABASE_URL'):
    raise ValueError("DATABASE_URL not found in environment variables")

print(f"Database URL loaded: {os.getenv('DATABASE_URL')[:20]}...")

# Initialize Prisma client
db = Prisma()

## Fetch Weather Data

Retrieve data from the `weather_db_v2` table (includes pressure)

In [3]:
# Connect and fetch data
await db.connect()
rows = await db.weather_db_v2.find_many(order={'timestamp': 'asc'})
await db.disconnect()

print(f"Retrieved {len(rows)} weather records")

EngineConnectionError: Could not connect to the query engine

## Prepare Data

Convert to DataFrame and set timezone-aware timestamps

In [ ]:
# Convert to DataFrame
df = pd.DataFrame([r.model_dump() for r in rows])
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Get local timezone
local_tz = datetime.now().astimezone().tzinfo
df['timestamp'] = df['timestamp'].dt.tz_convert(local_tz)

df.head()

## Hourly Buckets

Group data by hour and calculate averages

In [ ]:
# Add hour column (floor timestamp to hour)
df['hour'] = df['timestamp'].dt.floor('H')

# Group by hour and calculate averages
hourly = df.groupby('hour').agg({
    'temperature': 'mean',
    'humidity': 'mean',
    'pressure': 'mean'
}).reset_index()

hourly = hourly.sort_values('hour')
print(f"Hourly buckets: {len(hourly)}")
hourly.tail(3)

## Calculate Hourly Trends

Compare last completed hour vs previous hour

In [ ]:
def format_delta(value, unit, decimals=1):
    """Format a delta value with sign and unit"""
    sign = '+' if value >= 0 else ''
    return f"{sign}{value:.{decimals}f}{unit}"

def get_hourly_trend(hourly_df):
    """Calculate trend between last completed hour and previous hour"""
    if len(hourly_df) < 2:
        return None
    
    # Get current time
    now = datetime.now(hourly_df['hour'].iloc[-1].tzinfo)
    current_hour = now.replace(minute=0, second=0, microsecond=0)
    
    # Find last completed hour (the hour before current)
    last_completed_hour = current_hour - timedelta(hours=1)
    
    # Filter to only completed hours
    completed = hourly_df[hourly_df['hour'] < current_hour]
    
    if len(completed) < 2:
        return None
    
    # Get last two completed hours
    last_hour = completed.iloc[-1]
    prev_hour = completed.iloc[-2]
    
    # Calculate deltas
    temp_delta = last_hour['temperature'] - prev_hour['temperature']
    hum_delta = last_hour['humidity'] - prev_hour['humidity']
    pressure_delta = last_hour['pressure'] - prev_hour['pressure']
    
    return {
        'last_hour_time': last_hour['hour'],
        'prev_hour_time': prev_hour['hour'],
        'temp_delta': temp_delta,
        'temp_formatted': format_delta(temp_delta, '°C'),
        'humidity_delta': hum_delta,
        'humidity_formatted': format_delta(hum_delta, '%'),
        'pressure_delta': pressure_delta,
        'pressure_formatted': format_delta(pressure_delta, ' hPa'),
        'last_hour_values': {
            'temperature': last_hour['temperature'],
            'humidity': last_hour['humidity'],
            'pressure': last_hour['pressure']
        },
        'prev_hour_values': {
            'temperature': prev_hour['temperature'],
            'humidity': prev_hour['humidity'],
            'pressure': prev_hour['pressure']
        }
    }

trend = get_hourly_trend(hourly)
trend

## Display Results

Show formatted trend comparison

In [ ]:
if trend:
    print("="*60)
    print("HOURLY TREND ANALYSIS")
    print("="*60)
    print(f"\nPrevious Hour: {trend['prev_hour_time'].strftime('%Y-%m-%d %H:%M %Z')}")
    print(f"Last Hour:     {trend['last_hour_time'].strftime('%Y-%m-%d %H:%M %Z')}")
    print("\n" + "-"*60)
    print("\nTREND CHANGES:")
    print(f"  🌡️  Temperature: {trend['temp_formatted']}")
    print(f"  💧 Humidity:    {trend['humidity_formatted']}")
    print(f"  🔽 Pressure:    {trend['pressure_formatted']}")
    print("\n" + "-"*60)
    print("\nDETAILS:")
    print(f"\nPrevious Hour ({trend['prev_hour_time'].strftime('%H:%M')}):")
    print(f"  Temperature: {trend['prev_hour_values']['temperature']:.1f}°C")
    print(f"  Humidity:    {trend['prev_hour_values']['humidity']:.1f}%")
    print(f"  Pressure:    {trend['prev_hour_values']['pressure']:.1f} hPa")
    print(f"\nLast Hour ({trend['last_hour_time'].strftime('%H:%M')}):")
    print(f"  Temperature: {trend['last_hour_values']['temperature']:.1f}°C")
    print(f"  Humidity:    {trend['last_hour_values']['humidity']:.1f}%")
    print(f"  Pressure:    {trend['last_hour_values']['pressure']:.1f} hPa")
    print("\n" + "="*60)
else:
    print("⚠️  Insufficient data for trend analysis (need at least 2 completed hours)")

## Visualization (Optional)

Plot the hourly trends over time

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 10), sharex=True)

# Temperature
ax1.plot(hourly['hour'], hourly['temperature'], marker='o', color='tab:red', linewidth=2)
ax1.set_ylabel('Temperature (°C)', fontsize=12)
ax1.set_title('Hourly Weather Trends', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
if trend:
    ax1.axvline(trend['prev_hour_time'], color='gray', linestyle='--', alpha=0.5, label='Prev Hour')
    ax1.axvline(trend['last_hour_time'], color='black', linestyle='--', alpha=0.7, label='Last Hour')
    ax1.legend()

# Humidity
ax2.plot(hourly['hour'], hourly['humidity'], marker='o', color='tab:blue', linewidth=2)
ax2.set_ylabel('Humidity (%)', fontsize=12)
ax2.grid(True, alpha=0.3)
if trend:
    ax2.axvline(trend['prev_hour_time'], color='gray', linestyle='--', alpha=0.5)
    ax2.axvline(trend['last_hour_time'], color='black', linestyle='--', alpha=0.7)

# Pressure
ax3.plot(hourly['hour'], hourly['pressure'], marker='o', color='tab:green', linewidth=2)
ax3.set_ylabel('Pressure (hPa)', fontsize=12)
ax3.set_xlabel('Time', fontsize=12)
ax3.grid(True, alpha=0.3)
if trend:
    ax3.axvline(trend['prev_hour_time'], color='gray', linestyle='--', alpha=0.5)
    ax3.axvline(trend['last_hour_time'], color='black', linestyle='--', alpha=0.7)

# Format x-axis
ax3.xaxis.set_major_formatter(mdates.DateFormatter('%b %d\n%H:%M'))
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()